This script parses the PlantSeg LateralRootPrimordia dataset that we used in our paper. It comes in HDF5 format, and many have an 'ignore' label that will not work with Omnipose without some special revisions. We tossed out any data with this -1 label, but kept the files in their 'test' or 'train' folders. The images were downsampled by 1/3 to get cells in the range of 20-30px in diameter. This was done only to speed up training and evaluation with far smaller volumes. 

In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import time, os, sys

import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from scipy.ndimage import binary_closing,grey_closing
from skimage import measure
import napari 
import omnipose
import ncolor 
from omnipose.utils import sinebow

from scipy.ndimage import gaussian_filter
from omnipose.utils import normalize99, rescale
from scipy.ndimage import gaussian_filter

def localnormalize(im,sigma1=2,sigma2=3):
    im = normalize99(im)
    blur1 = gaussian_filter(im,sigma=sigma1)
    num = im - blur1
    blur2 = gaussian_filter(num*num, sigma=sigma2)
    den = np.sqrt(blur2)
    
    return normalize99(num/den+1e-8)

def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b

import tifffile
from omnipose import core
import cellpose_omni.core
use_GPU = cellpose.core.use_gpu()

In [ ]:
from cellpose_omni import io
save0 = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/'
export = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_good'
export = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small'
export = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small_all'
io.check_dir(export)
savedir = []
expdir = []
cpdir = []
ext = '.tif'
subset = ['test','train']
for s in subset:
    sd = os.path.join(save0,s)
    savedir.append(sd)
    sd = os.path.join(export,s)
    io.check_dir(sd)
    expdir.append(sd)
    sd = os.path.join(export,'cellpose',s)
    io.check_dir(sd)
    cpdir.append(sd)
savedir,expdir,cpdir

In [ ]:
# Make a function that extracts the file name 
from pathlib import Path
def getname(path,suffix=''):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

In [ ]:
#prepare to read from the hd5 files 
import h5py, fastremap
d = []
names = []
ext = '.tif'
# bad = [4,6,10,12,14,16,18,20]

In [ ]:
import scipy
do_all = True # training was done on the limited dataset; testing done on full, ignoring other regions?
downsize = True
factor = 1/3

In [ ]:
for folder,newfolder in zip(savedir,expdir):
    image_names = io.get_image_files(folder,extensions=['h5'])
    for filename in image_names:
        with h5py.File(filename, "r") as f:
            name = getname(filename)
            # if not np.any(["%04d" % b in name for b in bad]):

            # List all groups
            print("Keys: %s" % f.keys())

            masks = np.array(f['label'])            
            # if do_all:
            #     masks[masks<1] = 0 # just set all ignore regions to zero
            print(masks.min(),name)
            if masks.min()==1 or do_all:
                masks = omnipose.utils.format_labels(masks,ignore=do_all) # this corrects for any duplicate labes on distinct cells
                img = np.array(f['raw'])
                fastremap.renumber(masks,in_place=True)
                labels = fastremap.refit(masks)
 
                if downsize:
                    labels = scipy.ndimage.zoom(labels, factor, order=0)
                    img = scipy.ndimage.zoom(img, factor, order=1)
                    
                # tifffile.imwrite(os.path.join(newfolder,name+ext),np.uint8(omnipose.utils.normalize99(img)*(2**8-1)))
                tifffile.imwrite(os.path.join(newfolder,name+ext),img)
                tifffile.imwrite(os.path.join(newfolder,name+'_masks'+ext),labels)
                d.append(omnipose.core.diameters(labels))
                # mlist.append(masks)
                names.append(name)
           

The above files will be used for Omnipose in 3D. To train Cellpose, we have to slice these volumes along each axis. This will produce a lot of images. I'll just be doing it on the AWS server prior to training. Turns out this is 3,070 training images. 

In [ ]:
d = 3
idx = np.arange(d)
c = np.array([0]*(d-2)+[1]*2)
cyclic_perm(c),idx,c
ext = '.tif'
a = 'zyx'
# for basedir,save0 in zip(expdir,cpdir):
for basedir,save0 in zip([expdir[1]],[cpdir[1]]): #only do train 

    mask_filter = '_masks'
    img_names = io.get_image_files(basedir,mask_filter)
    mask_names,_ = io.get_label_files(img_names, mask_filter)

    for p1,p2 in zip(img_names,mask_names):
        img = tifffile.imread(p1)
        mask = tifffile.imread(p2)
        s = img.shape
        name = getname(p1)

        for inds,i in zip(cyclic_perm(c),idx):
            for k in range(s[i]):
                slc = tuple([slice(-1) if i else k for i in inds])
                suffix = '_'+a[i]+'{:03d}'.format(k)
                l = omnipose.utils.format_labels(mask[slc],clean=True) # this corrects for any duplicate labels on distinct cells
                if np.any(l): # only keep slices that have masks
                    tifffile.imwrite(os.path.join(save0,name+suffix+ext),img[slc])
                    tifffile.imwrite(os.path.join(save0,name+suffix+'_masks'+ext),l)

### Load data

In [ ]:
imgs = []
masks_gt = []
k = 0 #just test files now, switch to 1 to look at training set
for basedir,save0 in zip([expdir[k]],[cpdir[k]]): 
    mask_filter = '_masks'
    img_names = io.get_image_files(basedir,mask_filter)
    mask_names = io.get_label_files(img_names, mask_filter)

    for p1,p2 in zip(img_names,mask_names):
        imgs.append(tifffile.imread(p1))
        mgt = tifffile.imread(p2)
        mgt = omnipose.utils.format_labels(mgt,clean=True,verbose=False,ignore=do_all)
        masks_gt.append(mgt)
# count cells in test set
c = 0
for mgt in masks_gt:
    c+=len(fastremap.unique(mgt))-1 #ignore 0

In [ ]:
print('number of cells in {} is {}, number of images is {}'.format(subset[k],c,len(imgs)))

In [ ]:
%matplotlib inline
plt.imshow(masks[masks.shape[0]//2])

In [ ]:
# plt.imshow(img[img.shape[0]//2])
# img.shape,masks.shape
604

In [ ]:
# save slices theough each axis for figures
%matplotlib inline
from omnipose.utils import rescale, sinebow
from cellpose_omni import plot

cmap = plt.get_cmap(name='viridis')
x = np.linspace(0,1,1000)
colors=cmap(1-x)
colors[...,-1] = x
cmap = {
    'colors': colors,
    'name': 'custom',
    'interpolation': 'linear'
}
cmap = ListedColormap(list(colors))

save0 = os.path.join('/home/kcutler/DataDrive/omnipose_paper/Figure 7','slices',subset[k])
io.check_dir(save0)

fig = plt.figure(figsize=[20]*2)
def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b

for i,name in enumerate([names[0]]):
    print(name,i)
    for j in range(len(imgs)):
        img = omnipose.utils.normalize99(imgs[j]**.2)
        mgt = masks_gt[j]
        mnc = ncolor.label(mgt)
        smap = sinebow(mnc.max()+1)
        smap = ListedColormap(list(smap.values()))
        # plot flow and dist slices 
        d = img.ndim
        c = np.array([0]*(d-2)+[1]*2)

        slices = []
        idx = np.arange(d)
        for inds in cyclic_perm(c):
            slc = tuple([slice(None) if i else img.shape[k]//2 for i,k in zip(inds,idx)])
            # mgtnc = ncolor.label(mgt[slc])
            mgtnc = mnc[slc]
            
            pic = smap(mgtnc)
            pic[...,-1] = mgtnc>0
            tifffile.imwrite(os.path.join(save0,'slice_'+str(j)+'_'+str(inds)+'_gt.tif'),np.uint8(omnipose.utils.rescale(pic)*(2**8-1)))
            plt.imshow(mgt[slc],interpolation='none')
            pic = cmap(img[slc])
            # pic[...,-1] = imgs[j][slc]
            tifffile.imwrite(os.path.join(save0,'slice_'+str(j)+'_'+str(inds)+'_img.tif'),np.uint8(omnipose.utils.rescale(pic)*(2**8-1)))
            # plt.imshow(pic)
            # plt.imshow(mnc[slc],interpolation='none')
            plt.axis('off')
            plt.show()

### Plot in 3D

In [ ]:
# im = imgs[0]
# mask = masks_gt[0]
im = tiffile.imread('/Volumes/DataDrive/plantseg/traintest/LateralRootPrimordia/export/train/Movie3_T00004_crop_gt.tif')
mask = tiffile.imread('/Volumes/DataDrive/plantseg/traintest/LateralRootPrimordia/export/train/Movie3_T00004_crop_gt_masks.tif')

dim = mask.ndim
# fgt = omnipose.core.labels_to_flows(mask,dim=dim)
# dt = fgt[1]
# mu = fgt[2:2+dim]
# sd = fgt[-1]

_, dt, sd, mu = omnipose.core.masks_to_flows(mask,use_gpu=True,dim=dim,omni=True)

In [ ]:
viewer = napari.view_image(omnipose.utils.rescale(sd), name='distance',gamma=0.2,attenuation=0.05,depiction='volume')
viewer.add_image(omnipose.utils.rescale(dt==1), name='boundary',visible=True)
mnc = ncolor.label(mask,max_depth=11)
viewer.add_labels(mnc, name='labels_gt',visible = False, color=sinebow(mnc.max()))

In [ ]:
from cellpose_omni import plot
rgb = plot.dx_to_circ(mu,transparency=0)
# for k,s in enumerate('RGB'):
#     viewer.add_image(rgb[...,k], name=s,visible=True)
viewer.add_image(rgb, channel_axis=-1, name=['R','G','B']) #, colormap=['red', 'green', 'blue']

In [ ]:
# cmaps = ['Reds','Greens','Blues']
# x = np.linspace(0,1,100)
# z = np.zeros_like(x)
# o = np.ones_like(x)
# o[0] = 0
# red = np.stack((x,z,z,o),axis=-1)
# green = np.stack((z,x,z,o),axis=-1)
# blue = np.stack((z,z,x,o),axis=-1)
# colors = [red,green,blue]
# for k,s in enumerate('RGB'):
#     # cmap = plt.get_cmap(name=cmaps[k])
#     # # colors=cmap(1-x)
#     # colors = cmap(x)
#     # colors[...,-1] = x
    
#     new_colormap = {
#         'colors': colors[k],
#         'name': 'custom',
#         'interpolation': 'linear'
#     }
    
#     viewer.layers[s].colormap = new_colormap
#     # viewer.layers[s].colormap = 'gray_r'
#     # viewer.layers[s].rendering= 'translucent'
#     # # viewer.layers[s].contrast_limits=[0,1]
#     # viewer.layers[s].blending = 'additive'
#     # viewer.layers[s].opacity = 1
#     # viewer.layers[s].attenuation = .1

In [ ]:
# # plt.get_cmap(name=cmaps[k])
# # np.stack([c in rgb],axis=-1)
# viewer.add_image(rgb, name='rgb',visible=True,rgb=True)

In [ ]:
viewer.dims.ndisplay = 3
viewer.camera.center = [s//2 for s in im.shape]
viewer.camera.zoom=1.75
viewer.camera.angles=(49.01724873267338, -59.486485045019705, 36.815397248657874)
viewer.camera.perspective=0.0
viewer.camera.interactive=True

In [ ]:
cmap = plt.get_cmap(name='magma')
x = np.linspace(0,1,100)
colors=cmap(1-x)
# colors = cmap(x)
colors[...,-1] = x
# colors[0,...,-1] = 0
new_colormap = {
    'colors': colors,
    'name': 'custom',
    'interpolation': 'linear'
}
for key in ['distance']:
    viewer.layers[key].colormap = new_colormap
    viewer.layers[key].rendering='translucent'
    viewer.layers[key].contrast_limits=[0,1]
    
cmap = plt.get_cmap(name='inferno')
x = np.linspace(0,1,100)
colors=cmap(1-x)
# colors = cmap(x)
colors[...,-1] = x
new_colormap = {
    'colors': colors,
    'name': 'custom',
    'interpolation': 'linear'
}

key = 'boundary'
viewer.layers[key].colormap = new_colormap
viewer.layers[key].rendering='attenuated_mip'
viewer.layers[key].contrast_limits=[0,1]

In [ ]:
# add axes
n_axes = 3
vectors = np.zeros((n_axes, 2, 3), dtype=np.float32)
x_px = np.array([1, 0, 0])
y_py = np.array([0, 1, 0])
z_pz = np.array([0, 0, 1])
vectors[0, 1] = x_px
vectors[1, 1] = y_py
vectors[2, 1] = z_pz
for k in range(im.ndim):
    viewer.add_vectors(vectors[k], name='ax'+str(k), edge_width=1,edge_color='k',blending='translucent_no_depth', length=im.shape[k])

In [ ]:
basedir = '/home/kcutler/DataDrive/omnipose_paper/Figure 7/gt'
io.check_dir(basedir)
layerlist = ['distance','boundary','labels_gt']
for name in layerlist:
    viewer.layers[name].visible = False

for name in layerlist:
    viewer.layers[name].visible = True

    img = viewer.screenshot(size=(2000,2000),scale=1,canvas_only=True,flash=False)
    viewer.layers[name].visible = False

    io.imsave(os.path.join(basedir,'3D_'+name+'_seg.png'),img)

In [ ]:
# Show Euler integration 
dists = dt.copy()
dists[dists==0]=-5
cell_ID = 1
inds = np.array(np.nonzero(mask==cell_ID)).astype(np.int32)
masks_pred, p, tr = omnipose.core.compute_masks(mu,dists,inds=inds,calc_trace=True,use_gpu=1,
                                           omni=1,niter=200,interp=1,mask_threshold=-1,verbose=True,
                                          flow_threshold = 0.0, min_size=0, dim=3, cluster=1)

In [ ]:
# np.unique(masks_pred)
# tr.shape
bd_inds = np.argwhere(dists[tuple(inds)]==1) # which of the inds correspond to bpundary points

In [ ]:
# bd_inds = np.array(np.nonzero(dists[tuple(inds)]==1))
bd_inds = np.hstack(np.argwhere(dists[tuple(inds)]==1)) #equivalent, better shape
tr[:,bd_inds,:].shape

In [ ]:
bd_inds

In [ ]:
tracks = []
# for ID in bd_inds:
for ID in np.random.choice(bd_inds,len(bd_inds)//1):

    track = tr[:,ID,:].T
    times = np.arange(0,points.shape[0])[...,np.newaxis]
    IDs = np.ones_like(times)*ID
    
#      # calculate the speed as a feature
#     gz = np.gradient(track[:, 2])
#     gy = np.gradient(track[:, 3])
#     gx = np.gradient(track[:, 4])

#     speed = np.sqrt(gx ** 2 + gy ** 2 + gz ** 2)
#     distance = np.sqrt(x ** 2 + y ** 2 + z ** 2)

#     track[:, 5] = gz
#     track[:, 6] = gy
#     track[:, 7] = gx
#     track[:, 8] = speed
#     track[:, 9] = distance
    
    
    tracks.append(np.concatenate([IDs,times,track],axis=-1))
tracks = np.concatenate(tracks, axis=0)
features = {
    'time': tracks[:, 1]
}
    #     'gradient_z': tracks[:, 5],
    #     'gradient_y': tracks[:, 6],
    #     'gradient_x': tracks[:, 7],
    #     'speed': tracks[:, 8],
    #     'distance': tracks[:, 9],
    # }

In [ ]:
viewer.add_tracks(tracks,colormap='magma',features=features, color_by='time', blending='translucent')


In [ ]:
img = viewer.screenshot(size=(4000,)*2,scale=1,canvas_only=True,flash=False)

In [ ]:
basedir = '/home/kcutler/DataDrive/omnipose_paper/Figure 7/tracks'
io.check_dir(basedir)
io.imsave(os.path.join(basedir,'3D_'+str(cell_ID)+'_tracks.png'),img)

In [ ]:
viewer.add_labels(masks_pred, name='single',visible = False)

In [ ]:
m = np.zeros_like(dists,dtype=int)
m[tuple(inds)] = 1
viewer.add_labels(m, name='single_gt',visible = False)

In [ ]:
%matplotlib widget
# plt.imshow(masks_pred[masks_pred.shape[0]//2])
plt.imshow(mask[mask.shape[0]//2])

# viewer = napari.view_image(omnipose.utils.rescale(sd), name='distance',gamma=0.2,attenuation=0.05,depiction='volume')
# mnc = ncolor.label(masks_pred,max_depth=11)
# viewer.add_labels(mnc, name='labels_pred',visible = False, color=sinebow(mnc.max()))

### Run Cellpose3D

In [ ]:
# Cellpose seems to need training with  --chan 2 --chan2 1 
modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/cellpose/train/models/cellpose_residual_on_style_on_concatenation_off_train_2022_05_09_01_43_03.829853_epoch_499'
omni = False
rescale = False
diam_mean = 0
from cellpose_omni import models, core
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean)


In [ ]:
mask_threshold = 0 ##############
net_avg = 0
verbose = 0 #####

tile = 0
chans = [0,0]
# chans = None
compute_masks = 1
# n = [0]
rescale = None

masks_cp, flows_cp, _ = model.eval(imgs,channels = chans,rescale=rescale,
                                  mask_threshold=mask_threshold, net_avg=net_avg,
                                  transparency=True, flow_threshold=0., verbose=verbose,tile=1,
                                  compute_masks=compute_masks, do_3D=True, omni=False)

In [ ]:
# now do Omnipose

#best
modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_1951'
# no improvement over cp somehow 
# modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_2951'
# try stepping back; now a ral improvement over 1951
modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_2901'
# 2901 is great, 3051 is ok but not as strong, so try 3001 - also not as good as 1951
# modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_3001'
#3551 also not great, nor is 3501, but 3401 is terrific, 3451 awful, 3201 meh despite low loss, 3351 bad, 3201 bad, 3251 pretty good, 3151 not great,3301 bad,
# modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_3401'
# modeldir = '/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_small/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_08_01_56_56.310115_epoch_3301'

dim = 3
nclasses = dim+2
nchan = 1
omni = 1
rescale = False
diam_mean = 0
from cellpose_omni import models, core
# use_GPU = core.use_gpu(3)
# print('>>> GPU activated? %d'%use_GPU)
use_GPU = 1
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean, nclasses=nclasses, dim=dim, nchan=nchan)#,device=torch.device('cuda:1'))
model.pretrained_model

In [ ]:
mask_threshold = -5 ##############
diam_threshold = 12
net_avg = 0
cluster = 0 ##########
verbose = 1
tile = 0
chans = None
compute_masks = 1
resample=False
rescale=None
omni=True


masks_om, flows_om, _ = model.eval(imgs,channels=None,rescale=rescale,mask_threshold=mask_threshold,net_avg=net_avg,
                                  transparency=True,flow_threshold=0.,omni=omni,resample=resample,verbose=verbose,
                                  diam_threshold=diam_threshold,cluster=cluster,tile=0,
                                  compute_masks=compute_masks,flow_factor=10)
#     else:

In [ ]:
from cellpose_omni import io
basedir = '/home/kcutler/DataDrive/omnipose_paper/Figure 7'
io.check_dir(basedir)

masks = [masks_cp,masks_om]
flows = [flows_cp,flows_om]
names = ['cellpose','omnipose']
np.array(masks).shape

In [ ]:
# now quanititative comparison 
from stardist import matching
a = 0.5
b = 1.0
x = np.arange(a,b,(b-a)/100)
fig = plt.figure()
c = ['r','g']
JI = []


# confogure plot
sz = 1
golden = (1 + 5 ** 0.5) / 2
labelsize = 7

n = len(names)
z = 1
master_color_scheme = [[i,0,0] for i in np.linspace(1,.5,z)]+[[i,i,i] for i in np.linspace(.75,.25,n-z)]

darkmode = 0
if darkmode:
    plt.style.use('dark_background')
    axcol = 'w'
    colors = sinebow(n+1)
    colors = [colors[j+1] for j in range(n)]
    background_color = 'k'
    suffix = '_dark'
else:
    mpl.rcParams.update(mpl.rcParamsDefault)
    axcol = 'k'
    colors = master_color_scheme
    # colors = sinebow(n+1)
    # colors = [colors[j+1] for j in range(n)]
    background_color = np.array([1,1,1,1])
    suffix = ''

mpl.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(sz, sz)) 
ax = plt.axes()

plt.xticks(np.arange(min(x), max(x)+1, .25))
plt.xlim([0.5,1])
plt.ylim([0,.5])


for i,name in enumerate(names):
    met =[matching.matching(mgt,m, thresh=thresh) for mgt,m in zip(masks_gt, masks[i])]
    a = np.array([[met[k][i].accuracy for i in range(len(thresh))] for k in range(len(met))]).T
    np.save(os.path.join(basedir,name+'_3D_plants_individual'),a)
    a = np.mean(a,axis=1)
    JI.append(a)
    plt.plot(x,a, label=name,c=colors[i],linewidth=.5)#+['masks_cp'])
    np.save(os.path.join(basedir,name+'cellpose_3D_plants'),a)
    # plt.legend
    
# ax.legend(prop={'size': labelsize}, loc='upper left', frameon=False,bbox_to_anchor=(1.05, 1))
ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)

ax.set_ylabel('Jaccard Index', fontsize = labelsize)
ax.set_xlabel('IoU matching threshold', fontsize = labelsize)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)

fig.patch.set_facecolor(background_color)
plt.show()

figname = 'JI_vs_IoU'
for ext in ['.pdf','.png','.eps']:
    fig.savefig(os.path.join(basedir,figname+suffix+ext),bbox_inches="tight",pad_inches = 0.05)


In [ ]:
cutoff = 80 # IoU cutoff of 0.8

cp = JI[0].T
om = JI[1].T

av_all = np.mean(np.divide((om-cp),cp,out=np.zeros_like(cp), where=np.logical_and(cp!=0,~np.isnan(cp))))
av_up = np.mean(np.divide((om[cutoff:]-cp[cutoff:]),cp[cutoff:],out=np.zeros_like(cp[cutoff:]), where=np.logical_and(cp[cutoff:]!=0,~np.isnan(cp[cutoff:]))))
av_dwn = np.mean(np.divide((om[:cutoff]-cp[:cutoff]),cp[:cutoff],out=np.zeros_like(cp[:cutoff]), where=np.logical_and(cp[:cutoff]!=0,~np.isnan(cp[:cutoff]))))

print(av_all*100,av_up*100,av_dwn*100,(np.mean(om)-np.mean(cp))/np.mean(cp)*100) # turn

In [ ]:
import random
random.seed(42)

k = 0
viewer = napari.view_image(imgs[k], name='cells',visible = False)
viewer.dims.ndisplay = 3

mnc = ncolor.label(masks_gt[k],max_depth=11)
viewer.add_labels(mnc, name='labels_gt',visible = False, color=sinebow(mnc.max()))


# cmap = sinebow(masks[0][k].max())
# l = list(cmap.items())[1:]
# random.shuffle(l)
# cmap = [(0,cmap[0])]+l
# cmap = dict(cmap)
mnc = ncolor.label(masks[0][k],max_depth=20)
cmap =  sinebow(mnc.max())
viewer.add_labels(mnc, name=names[0],visible = False, color=cmap)



mnc = ncolor.label(masks[1][k],max_depth=11)
viewer.add_labels(mnc, name=names[1], color=sinebow(mnc.max()))

In [ ]:
viewer.camera.center = [s//2 for s in imgs[k].shape]
viewer.camera.zoom=2
viewer.camera.angles=(49.01724873267338, -59.486485045019705, 36.815397248657874)
viewer.camera.perspective=0.0
viewer.camera.interactive=True

In [ ]:
k = 0
j = 1
img = imgs[k]
n_axes = 3
vectors = np.zeros((n_axes, 2, 3), dtype=np.float32)
x_px = np.array([1, 0, 0])
y_py = np.array([0, 1, 0])
z_pz = np.array([0, 0, 1])
vectors[0, 1] = x_px
vectors[1, 1] = y_py
vectors[2, 1] = z_pz
for k in range(img.ndim):
    viewer.add_vectors(vectors[k], name='ax'+str(k), edge_width=1,edge_color='k',blending='translucent_no_depth', length=img.shape[k])

In [ ]:
# turn off all masks
layerlist = names+['labels_gt']
for name in layerlist:
    viewer.layers[name].visible = False

for name in layerlist:
    viewer.layers[name].visible = True

    img = viewer.screenshot(size=(2000,2000),scale=1,canvas_only=True,flash=False)
    viewer.layers[name].visible = False

    io.imsave(os.path.join(basedir,'3D_'+name+'_seg.tiff'),img)

In [ ]:
viewer.add_image(omnipose.utils.rescale(flows_om[0][2]), name='distance',gamma=0.2,attenuation=0.05,depiction='volume')

In [ ]:
cmap = plt.get_cmap(name='viridis')
x = np.linspace(0,1,100)
colors=cmap(1-x)
colors[...,-1] = x
new_colormap = {
    'colors': colors,
    'name': 'custom',
    'interpolation': 'linear'
}
key = 'distance'
viewer.layers[key].colormap = new_colormap
viewer.layers[key].rendering='attenuated_mip'
viewer.layers[key].contrast_limits=[.01,1]

In [ ]:
# Plot cross sections and save some output for the figure 
from matplotlib.colors import ListedColormap

cmap = mpl.cm.get_cmap('viridis')
for i,name in enumerate(names):
    print(name,i)
    for j in range(len(imgs)):
        img = imgs[j]
        mgt = masks_gt[j]
        msk = masks[i][j]
        dt = flows[i][j][2]
        bd = flows[i][j][4]
        frgb = flows[i][j][0]
        k = img.shape[0]//2 # middle slice 
        fig = plt.figure()
        print(np.array(msk).shape,k)
        mgtnc = ncolor.label(mgt[k])
        plt.imshow(np.hstack(tuple(map(cmap,map(omnipose.utils.rescale,[img[k],dt[k], bd[k] ,
                                                                        ncolor.label(msk[k]),
                                                                        mgtnc])))
                             +(frgb[k]/255,)))
        
        tifffile.imwrite(os.path.join(basedir,'slice_'+name+'_'+str(j)+'_frgb.tif'),frgb[k])
        if i:
            smap = sinebow(4)
            smap = ListedColormap(list(smap.values()))
            pic = smap(mgtnc)
            pic[...,-1] = mgtnc>0
            tifffile.imwrite(os.path.join(basedir,'slice_'+str(j)+'_gt.tif'),np.uint8(omnipose.utils.rescale(pic)*(2**8-1)))


        plt.axis('off')
        

In [ ]:
### Plot some ground truth 

In [ ]:
5

In [ ]:
from mpl_toolkits import mplot3d


fig = plt.figure()
ax = plt.axes(projection='3d')
ax.voxels(imgs[0])

In [ ]:
# np.unique(sd)
# plt.imshow(sd[-1])
import ipyvolume as ipv
# ipv.figure()
# ipv.quickvolshow(sd)

import ipyvolume.pylab as p3
p3.figure()
p3.style.use('light')
p3.style.axes_off()
p3.style.box_off()
# vol = p3.volshow(np.log(T+1e-12))
# p3.volshow(ncolor.label(seg))
# points = np.nonzero(dt==1)
# m = np.max(mask)-100
# points = np.nonzero((mask>m-5)*(dt==1)*(mask<=m))
# select = np.logical_or.reduce([mask == i for i in np.unique(mask[n])[1:]])
# points = np.nonzero((dt==1)*select)


# mg = np.meshgrid(*points,indexing='ij')
# ax.quiver(*mg,*tuple(mu),length=0.1)
# sub_mu = [mu[(i,)+points] for i in range(mu.shape[0])]
# ax.quiver(*mg,*sub_mu,normalize=True)
# sub_p =  [p[(i,)+points] for i in range(p.shape[0])]
# p3.scatter(*sub_p, size=1, marker="sphere",origin=np.mean(sub_p,axis=1))
vol = p3.volshow(np.log(imgs[0]+.1),lighting=0)
# ipv.quickvolshow(np.log(imgs[0]+1))
# vol.ray_steps =20


p3.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Generate nicely looking random 3D-field
np.random.seed(0)
vol = omnipose.utils.normalize99(imgs[0])
shape = vol.shape
print(shape,vol.min(),vol.max())
X, Y, Z = np.mgrid[:shape[0], :shape[1], :shape[2]]
vol = np.zeros((l, l, l))

fig = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=vol.flatten(),
    # isomin=0.0,
    # isomax=0.99,
    opacity=0.9,
    # surface_count=1,
    opacityscale=[[-0.5, 1], [-0.2, 0], [0.2, 0], [0.5, 1]],
    surface_count = 30
    ))
fig.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np
X, Y, Z = np.mgrid[-8:8:40j, -8:8:40j, -8:8:40j]
values = np.sin(X*Y*Z) / (X*Y*Z)

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.1,
    isomax=0.8,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()

In [ ]:
from mayavi import mlab
mlab.init_notebook()
# x, y, z = np.ogrid[-10:10:20j, -10:10:20j, -10:10:20j]
# s = np.sin(x*y*z)/(x*y*z)
# mlab.pipeline.volume(mlab.pipeline.scalar_field(omnipose.utils.normalize99(s)),vmin=0, vmax=0.8)
# mlab.contour3d(s)
mlab.test_plot3d()

In [ ]:
# plot some slices 
cmap = mpl.cm.get_cmap('viridis')
for i,name in enumerate(names):
    print(name,i)
    for j in range(len(imgs)):
        img = imgs[j]
        mgt = masks_gt[j]
        msk = masks[i][j]
        dt = flows[i][j][2]
        bd = flows[i][j][4]
        frgb = flows[i][j][0]
        k = img.shape[0]//2 # middle slice 
        fig = plt.figure()
        print(np.array(msk).shape,k)
        plt.imshow(np.hstack(tuple(map(cmap,map(omnipose.utils.rescale,[img[k],dt[k], bd[k] ,
                                                                        ncolor.label(msk[k]),
                                                                        ncolor.label(mgt[k])])))
                             +(frgb[k]/255,)))
        plt.axis('off')

In [ ]:
basedir

In [ ]:
# plt.imshow(msk)
msk.dtype

In [ ]:
im = tifffile.imread('/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_good/test/Movie1_t00030_crop_gt.tif')

In [ ]:
# viewer.axes.visible = False
# viewer.overlays.interaction_box.show_handle=1
viewer = napari.view_image(omnipose.utils.rescale(im), name='cells',gamma=0.2,attenuation=0.05,depiction='volume')
viewer.dims.ndisplay = 3
viewer.camera.center = [s//2 for s in im.shape]
viewer.camera.zoom=.7
viewer.camera.angles=(49.01724873267338, -59.486485045019705, 36.815397248657874)
viewer.camera.perspective=0.0
viewer.camera.interactive=True
n_axes = 3
vectors = np.zeros((n_axes, 2, 3), dtype=np.float32)
x_px = np.array([1, 0, 0])
y_py = np.array([0, 1, 0])
z_pz = np.array([0, 0, 1])
vectors[0, 1] = x_px
vectors[1, 1] = y_py
vectors[2, 1] = z_pz
for k in range(im.ndim):
    viewer.add_vectors(vectors[k], name='ax'+str(k), edge_width=1,edge_color='k',blending='translucent_no_depth', length=im.shape[k])

In [ ]:
viewer.layers[0].contrast_limits=[.002,1]


In [ ]:

# colors = np.linspace(
#     start=[1, 1, 1, 0],
#     stop=[0, 0, 0, 1],
#     num=10,
#     endpoint=True
# )
cmap = plt.get_cmap(name='viridis')
x = np.linspace(0,1,100)
colors=cmap(1-x)
colors[...,-1] = x
new_colormap = {
    'colors': colors,
    'name': 'custom',
    'interpolation': 'linear'
}
viewer.layers[0].colormap = new_colormap
viewer.layers[0].rendering='attenuated_mip'
viewer.layers[0].contrast_limits=[.01,1]

# k = 0
# im = imgs[k]

In [ ]:
img = viewer.screenshot(size=(2000,2000),scale=1,canvas_only=True,flash=False)
from cellpose_omni import io
basedir = '/home/kcutler/DataDrive/omnipose_paper/Figure 7'
io.check_dir(basedir)
io.imsave(os.path.join(basedir,'3D.tiff'),img)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.rcParams['figure.dpi'] = 300
plt.imshow(img)
plt.axis('off')

In [ ]:
viewer.__dict__

In [ ]:
len(masks_gt)

In [ ]:
cmap = mpl.cm.get_cmap('viridis')

for j in range(len(imgs)):
    img = imgs[j]
    msk = masks[j]
    dt = flows[j][2]
    bd = flows[j][4]
    frgb = flows[j][0]
    k = img.shape[0]//2 # middle slice 
    fig = plt.figure()
    plt.imshow(np.hstack(tuple(map(cmap,map(omnipose.utils.rescale,[img[k],dt[k],bd[k],
                                                                    ncolor.label(msk[k]),
                                                                    ncolor.label(masks_gt[j][k])])))
                         +(frgb[k]/255,)))
    plt.axis('off')

In [ ]:
# now quanititative comparison 

from stardist import matching
a = 0.5
b = 1.0
thresh =np.arange(a,b,(b-a)/100)
met =[matching.matching(mgt,m, thresh=thresh) for mgt,m in zip(masks_gt, masks)]
a = np.array([[met[k][i].accuracy for i in range(len(thresh))] for k in range(len(met))]).T
plt.plot(thresh,a)#+['masks_cp'])
np.save(os.path.join(basedir,'omnipose_3D_plants_individual'),a)
a = np.mean(a,axis=1)
np.save(os.path.join(basedir,'omnipose_3D_plants'),a)


In [ ]:
names = ['cellpose','omnipose']
JI = [np.array(np.load(os.path.join(basedir,name+'_3D_plants_individual.npy'))) for name in names]
JI_mean = [np.array(np.load(os.path.join(basedir,name+'_3D_plants.npy'))) for name in names]

In [ ]:
JI[0].shape

In [ ]:
plt.figure()
plt.plot(thresh,JI[0],label=names[0])#+['masks_cp'])
plt.plot(thresh,JI[1],label=names[1])#+['masks_cp'])
plt.legend()

In [ ]:
masks_all,

In [ ]:
plt.imshow(masks[0])

In [ ]:
# d = omnipose.core.diameters(masks_gt)
# d
image_names = io.get_image_files(savedir[1],extensions=['h5'])
filename = image_names[10]
with h5py.File(filename, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())
        masks = np.array(f['label'])
        print(masks.min())
        masks = omnipose.utils.format_labels(masks)
        name = getname(filename)
        print(name)
        fastremap.renumber(masks,in_place=True)
        labels = fastremap.refit(masks)
        img = np.array(f['raw'])

In [ ]:
img.shape

In [ ]:
import edt
dt = edt.edt(labels)

In [ ]:
# np.max(masks),np.max(labels),np.min(masks),np.min(labels),filename
omnipose.core.diameters(masks,dt)

In [ ]:
from omnipose.utils import rescale 
k = 100
plt.imshow(np.hstack(tuple(map(rescale,(img[:,k],ncolor.label(labels[:,k]),dt[:,k])))))
# plt.imshow(dt[:,500])

In [ ]:
6

In [ ]:
viewer = napari.view_image(img, name='cells')
viewer.add_labels(labels, name='labels')

In [ ]:
# len(flows)
a,b = map(list,zip(*[[l*2,l/2] for l in range(4)]))
a,b

In [ ]:
from omnipose.core import diameters
diameters(masks[slc])

In [ ]:
a = np.ones([10,10])
pad = 4
apad = np.pad(a,pad)
unpad = tuple([slice(pad,-pad) if pad else slice(None,None)]*a.ndim)
apad[unpad].shape,a.shape,apad.shape

In [ ]:
np.pad(a,None)

In [ ]:
import fastremap
m,_ = fastremap.renumber(masks,in_place=False)
np.max(m)

In [ ]:
import edt
dt = edt.edt(m[slc][50])
dt

In [ ]:
im.shape

In [ ]:
# modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_03_18_58_51.851134_epoch_301'
import tifffile
im = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC027/test/0000.tif')

In [ ]:
masks_gt = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC027/test/0000_masks.tif')
# d = omnipose.core.diameters(masks_gt)
# d
im.shape

In [ ]:
modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_03_18_58_51.851134_epoch_101'

modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_03_22_16_13.356647_epoch_2601'

dim = 3
nclasses = dim+2
nchan = 1
omni = 1
rescale = False
diam_mean = 0
from cellpose_omni import models, core
use_GPU = core.use_gpu()

# print('>>> GPU activated? %d'%use_GPU)
# use_GPU = 0 # uh oh, 3D models not working for CPU
# use_GPU = 0
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean, nclasses=nclasses, dim=dim, nchan=nchan)

In [ ]:
mask_threshold = -5 ##############
diam_threshold = 12
net_avg = 0
cluster = 1
verbose = 0 #####

tile = 0
chans = [0,0]
chans = None
compute_masks = 1
slc_crop = slice(None, -1, None)
# slc_crop = slice(100, 120, None)
# slc_crop = slice(13, 53, None) # 733
# slc_crop = slice(50, 100, None) # 704
# slc_crop = slice(15, 52, None) # 707
# slc_crop = slice(30, 70, None) # 712
# slc_crop = (Ellipsis,)+tuple([slice(0,129)]*2)
# slc_crop = (Ellipsis,)+tuple([slice(0,400)]*2)

resample=False
# rescale = .5 #None
rescale=None

imcrop = im[slc_crop]
# pad = 30
# imcrop = np.pad(imcrop,pad,mode='reflect')
# imcrop = im[100:120]
# imcrop = im.copy()
# import torch
#TILING not working in 3D, should fix that 
masks, flows, styles = model.eval(imcrop,channels = chans,rescale=rescale,mask_threshold=mask_threshold,net_avg=net_avg,
                                  transparency=True,flow_threshold=0.,omni=omni,resample=resample,verbose=verbose,
                                  diam_threshold=diam_threshold,cluster=cluster,tile=1,
                                  compute_masks=compute_masks,batch_size=2)
#     else:

In [ ]:
for i,a in enumerate(['a','b']):
    print(i,a)

In [ ]:
k = 50
dt = flows[2]
bd = flows[4]
cmap = mpl.cm.get_cmap('viridis')

plt.imshow(np.hstack(tuple(map(cmap,map(omnipose.utils.rescale,[imcrop[k],dt[k],bd[k],masks[k]])))+(flows[0][k]/255,)))

In [ ]:
from stardist import matching
a = 0.5
b = 1.0
thresh =np.arange(a,b,(b-a)/100)
met = [[]]*1
met[0] = matching.matching(masks_gt[slc_crop], newmasks, thresh=thresh)
# gotta make sure precision is jaccard index!!!!
plt.plot(thresh,np.array([[met[k][i].accuracy for i in range(len(thresh))] for k in range(len(met))]).T)#+['masks_cp'])
# plt.legend()

In [ ]:
imgi = im.copy()[np.newaxis]

In [ ]:
tile_overlap = 0.1
bsize = 224

nchan = imgi.shape[0]
shape = imgi.shape[1:]
dim = len(shape)
tile_overlap = min(0.5, max(0.05, tile_overlap))
# bsizeY, bsizeX = min(bsize, Ly), min(bsize, Lx)
# B = [np.int32(min(b,s)) for s,b in zip(im.shape,bsize)] if bzise variable
bbox = tuple([np.int32(min(bsize,s)) for s in shape])

# tiles overlap by 10% tile size
ntyx = [1 if s<=bsize else int(np.ceil((1.+2*tile_overlap) * (s / bsize))) for s in shape]
start = [np.linspace(0, s-b, n).astype(int) for s,b,n in zip(shape,bbox,ntyx)]

sub = [[] for n in range(dim)]
# IMG = np.zeros((len(ystart), len(xstart), nchan,  bsizeY, bsizeX), np.float32)
IMG = np.zeros(tuple([len(s) for s in start])+(nchan,)+bbox, np.float32)

# for j in range(len(ystart)):
#     for i in range(len(xstart)):
#         ysub.append([ystart[j], ystart[j]+bsizeY])
#         xsub.append([xstart[i], xstart[i]+bsizeX])
#         IMG[j, i] = imgi[:, ysub[-1][0]:ysub[-1][1],  xsub[-1][0]:xsub[-1][1]]
IMG.shape, bbox,ntyx, np.diff(start[1]),

In [ ]:
int(np.ceil((1.+2*tile_overlap) * (shape[1] / bsize)))

In [ ]:
start

In [ ]:
import itertools
intervals = [[slice(si,si+bsize) for si in s] for s in start]
# intervals = []
# for s in start:
#     intervals.append([slice(si,si+bsize) for si in s])
#     # if len(s)>1:
#     #     intervals.append([slice(si,si+bsize) for si in s])
#     # else:
    #     intervals.append([slice(None)])
subs = list(itertools.product(*intervals))
len(subs)

In [ ]:
I = []
for slc in subs:
    imcrop = imgi[(Ellipsis,)+slc]
    print(imcrop.shape)
    I.append(imcrop)

In [ ]:
I2 = np.stack(I)
I2.shape

In [ ]:
# viewer = napari.view_image(I[0], name='cells')
I[0].shape


In [ ]:
from cellpose_omni.transforms import make_tiles
IMG, ysub, xsub, Ly, Lx = make_tiles(im[0][np.newaxis])

In [ ]:
IMG.shape,Ly,Lx,ysub

In [ ]:
viewer = napari.view_image(imcrop, name='cells')
viewer.add_labels(masks, name='labels')

In [ ]:
# viewer.add_labels(masks_gt[slc_crop], name='labels_gt')
viewer.add_image(bd, name='bd')
viewer.add_image(dt, name='dt')

In [ ]:
viewer.add_labels(ncolor.label(masks_gt[slc_crop]), name='labels_gt')


In [ ]:
viewer.add_image(flows[0], name='flows')

In [ ]:
mu = flows[1]
dt = flows[2]
p = flows[3]
bd = flows[4]
hdbscan = 1
cluster = 0
newmasks,p,_,lab= omnipose.core.compute_masks(mu,dt,bd=bd,mask_threshold=mask_threshold,
                                           use_gpu=True,verbose=True,cluster=cluster, 
                                           flow_threshold=0., dim=3, hdbscan=hdbscan, 
                                              debug=1, eps=3, min_size=30, nclasses=4)

In [ ]:
alg = ['','H']
mnc = ncolor.label(newmasks)
viewer.add_labels(mnc,name='{}DBSCAN'.format(alg[hdbscan]),color=sinebow(mnc.max()))
# 17.91925573348999 for hdbscan, 10.16240119934082 for dbscan ; then 25 vs 9 later 

In [ ]:
np.unique(newmasks)
color = sinebow(newmasks.max())
# [color[l]]
selection = newmasks>0
sub_p = [p[i].ravel()[selection.ravel()] for i in range(p.shape[0])]
# sub_col = [color[l] for l in lab[selection.ravel()]]
sub_col = [color[l] for l in mnc[mnc>0]]
data = np.stack(sub_p,axis=-1)
viewer.add_points(np.round(data),size=1,symbol='disc',face_color=sub_col,edge_color=sub_col)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=400,min_samples=3)
labels = clusterer.fit_predict(np.array(sub_p).T)
len(np.unique(labels))

In [ ]:
clusterer = hdbscan.HDBSCAN(cluster_selection_epsilon=2**0.5,min_samples=3, allow_single_cluster=True)
clusterer.fit(np.stack(sub_p,axis=1))

In [ ]:
np.unique(clusterer.labels_)
# np.sqrt(2)

In [ ]:
clusterer.condensed_tree_.plot()

In [ ]:
from sklearn.cluster import DBSCAN
eps=1.33
db = DBSCAN(eps=eps, min_samples=5, n_jobs=-1).fit(np.array(sub_p).T)

In [ ]:
len(np.unique(db.labels_))

In [ ]:
# np.prod(p.shape)
# p[0].ravel().shape,(points.ravel()).shape
%timeit p[0].ravel()[points.ravel()]
# p[0][points].ravel()

In [ ]:
%timeit p[0][points].ravel()

In [ ]:
# import datashader as ds, pandas as pd, colorcet
# !pip install vaex-jupyter
import vaex
# df = vaex.from_arrays(z=p[(0,+)],y=p[1],x=p[2])

In [ ]:
axes = [i for i in 'zyx']
points = bd>0
pdict = dict(zip(axes, [p[i].ravel()[points.ravel()] for i in range(p.shape[0])]))

In [ ]:
df = vaex.from_dict(pdict)
df

In [ ]:
import vaex.jupyter.model as vjm
await vaex.jupyter.gather()

In [ ]:
# df.viz.scatter(df.x, df.y, selection=df.z==40, c="red", alpha=0.5, s=4)
%matplotlib inline
plt.figure()
df.viz.heatmap([["x", "y"], ["x", "z"], ["y", "z"]], title="Face on and edge on", figsize=(8, 4), limits='100%')

In [ ]:
df.viz.heatmap("z", "y", z="FeH:-3,-1,8",
               visual=dict(row="z"),
               figsize=(12, 8),
               f="log",
               wrap_columns=3,
               limits='99%');


In [ ]:
omni = 1
masks, dists, T, mu = omnipose.core.masks_to_flows(mask,use_gpu=1,omni=omni,dim=mask.ndim)

In [ ]:
from omnipose.core import divergence
from omnipose.utils import rescale
from omnipose import utils
from cellpose_omni import plot
from scipy.ndimage import gaussian_filter
mu_f = gaussian_filter(mu,sigma=.5)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=[20]*2)

if mask.ndim==2:
    plt.imshow(plot.dx_to_circ(mu))
else:
    # plot flow and dist slices 
    from omnipose.utils import rescale
    d = mu.shape[0]
    c = np.array([1]*2+[0]*(d-2))
    # c = np.arange(d)
    def cyclic_perm(a):
        n = len(a)
        b = [[a[i - j] for i in range(n)] for j in range(n)]
        return b
    slices = []
    idx = np.arange(d)
    cmap = mpl.cm.get_cmap('magma')
    # pic = cmap(np.sqrt(heat))
    # pic[:,:,-1] = masks>0
    for inds in cyclic_perm(c):
        slc = tuple([slice(-1) if i else mu.shape[k+1]//2 for i,k in zip(inds,idx)])
        flow = rescale(plot.dx_to_circ(mu_f[np.where(inds)+slc],transparency=1))
        # dist = cmap(rescale(T)[slc])
        vals = rescale(np.log(T+1)[slc]) if omni else rescale(np.log(T+1e-5)[slc])
        dist = cmap(vals)
        dist[:,:,-1]=vals
        
        
        vals = utils.normalize99(divergence(mu_f))[slc]
        div = cmap(vals)
        div[:,:,-1]=vals
        div = utils.normalize99(div)
        
        
        msk = rescale(np.stack([mask[slc]]*4,axis=-1))>0
        dist[:,:,-1]=mask[slc]>0
        fig = plt.figure(figsize=[80]*2)
        plt.imshow(np.hstack((msk,flow,dist,div)),interpolation='none')
        plt.axis('off')
        plt.show()

In [ ]:
from cellpose_omni import transforms
tyx = (50,)*3
rs = None
omni = 1
#[imgs[k][np.newaxis,:,:] if imgs[k].ndim==2 else imgs[k] for k in n], [flows[k] for k in n]
img,lbl,sc = transforms.random_rotate_and_resize([imcrop[np.newaxis]],[masks],
                                                 tyx=tyx,
                                                 omni=omni,
                                                 rescale=rs,
                                                 dim=masks.ndim)


In [ ]:

from omnipose.utils import rescale
from omnipose.core import sigmoid

j = 0
img0 = img[j].squeeze()
mask0 = lbl[j][0]
bin0 = lbl[j][1]
bd0 = lbl[j][2]
dist0 = lbl[j][3]
weight0 = lbl[j][4]
mu0 = lbl[j][5:]

d = mu0.shape[0]
c = np.array([1]*2+[0]*(d-2))
# c = np.arange(d)
def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b
slices = []
idx = np.arange(d)
cmap = mpl.cm.get_cmap('magma')

for inds in cyclic_perm(c):
    
    slc = tuple([slice(-1) if i else mu0.shape[k+1]//2 for i,k in zip(inds,idx)])
    flow = rescale(plot.dx_to_circ(mu0[np.where(inds)+slc],transparency=1))
    # dist = cmap(rescale(T)[slc])
    vals = rescale(dist0)[slc] if omni else rescale(np.log(dist0+1e-5)[slc])
    dist = cmap(vals)
    dist[:,:,-1]=vals


    vals = utils.normalize99(divergence(mu0))[slc]
    div = cmap(vals)
    div[:,:,-1]=vals
    div = utils.normalize99(div)
    
    bin_ = cmap(bin0[slc])
    bd = cmap(bd0[slc])
    weight = cmap(rescale(weight0)[slc])

    msk = cmap(rescale(ncolor.label(mask0[slc])))
    fig = plt.figure(figsize=[80]*2)
    plt.imshow(np.hstack((msk,bin_,flow,dist,bd,weight,div)),interpolation='none')
    plt.axis('off')
    plt.show()

In [ ]:
ncolor.label(mask0).shape

In [ ]:
plt.imshow(bd)

In [ ]:
lbl.shape

In [ ]:
mgood = tifffile.imread('/home/kcutler/DataDrive/plantseg/traintest/LateralRootPrimordia/export_good/test/Movie1_t00030_crop_gt_masks.tif')

In [ ]:
import fastremap
len(fastremap.unique(mgood))